<a href="https://www.kaggle.com/code/franciscosuarezb/crosswalk-detection?scriptVersionId=185986423" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
data_dir = '/kaggle/input/crosswalk-detection-dataset/Data'
images_dir = os.path.join(data_dir, 'images')
labels_dir = os.path.join(data_dir, 'labels')

In [ ]:
# Listar archivos de imágenes y labels
image_files = os.listdir(images_dir)
label_files = os.listdir(labels_dir)

print(f"Total imágenes: {len(image_files)}")
print(f"Total labels: {len(label_files)}")


In [ ]:
# Dividir en entrenamiento y prueba
train_images, test_images = train_test_split(image_files, test_size=0.15, random_state=42)
# Dividir el conjunto de entrenamiento en entrenamiento y validación
train_images, val_images = train_test_split(train_images, test_size=0.20, random_state=42)  # 0.25 * 0.8 = 0.2

print(f"Total entrenamiento: {len(train_images)}")
print(f"Total validación: {len(val_images)}")
print(f"Total prueba: {len(test_images)}")


In [ ]:
output_dir = '/kaggle/working/dataset'

# Crear directorios si no existen
os.makedirs(os.path.join(output_dir, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val/labels'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test/labels'), exist_ok=True)


In [ ]:
print(f'Train_images: {train_images[:3]}')
print(f'Train_images: {test_images[:3]}')
print(f'Train_images: {val_images[:3]}')

In [ ]:
# Crear una nueva lista con la extensión .txt
train_labels = [f.replace('.jpg', '.txt') for f in train_images]
val_labels=[f.replace('.jpg', '.txt') for f in val_images]
test_labels=[f.replace('.jpg', '.txt') for f in test_images]

print(f' Total Train_Labels:{len(train_labels)}')
print(f'Total Val_Labels:{len(val_labels)}')
print(f'Total test_labels:{len(test_labels)}')

In [ ]:
print(f'Train_Labels:{train_labels[:3]}')
print(f'Val_Labels:{val_labels[:3]}')
print(f'Test_Labels:{test_labels[:3]}')


In [ ]:
import os
import shutil

def move_files(file_list, src_dir, dst_dir):
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        dst_file = os.path.join(dst_dir, file_name)
        try:
            shutil.copy(src_file, dst_file)
        except FileNotFoundError:
            print(f"Archivo no encontrado: {src_file}. Creando un archivo .txt vacío.")
            if file_name.endswith('.txt'):
                open(dst_file, 'a').close()

# Mover archivos de entrenamiento
move_files(train_images, images_dir, os.path.join(output_dir, 'train/images'))
move_files(train_labels, labels_dir, os.path.join(output_dir, 'train/labels'))

# Mover archivos de validación
move_files(val_images, images_dir, os.path.join(output_dir, 'val/images'))
move_files(val_labels, labels_dir, os.path.join(output_dir, 'val/labels'))

# Mover archivos de prueba
move_files(test_images, images_dir, os.path.join(output_dir, 'test/images'))
move_files(test_labels, labels_dir, os.path.join(output_dir, 'test/labels'))


In [ ]:
for phase in ['train', 'val', 'test']:
    print(f"{phase} images: {len(os.listdir(os.path.join(output_dir, phase, 'images')))}")
    print(f"{phase} labels: {len(os.listdir(os.path.join(output_dir, phase, 'labels')))}")


In [ ]:
from datetime import datetime
from functools import wraps

import numpy as np
from IPython.lib.display import Audio
from ultralytics import YOLO


# Crear el yaml archivo de conf de yolov8

In [ ]:
yaml_content = """
path: /kaggle/working/dataset

train: train/images
val: val/images


nc: 1

names:
     - crosswalk
"""

# Guardar el archivo
with open('/kaggle/working/crosswalk_dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("Archivo .yaml creado con éxito.")


# Crear una carpeta para guardar el proyecto

In [ ]:
os.makedirs(os.path.join('/kaggle/working/', 'training_results'), exist_ok=True)


In [ ]:
proyect_results_path="/kaggle/working/training_results"

## Cargar los pesos pre-entrenados

In [ ]:
#%%
from ultralytics import YOLO

epochs = 150 
weight = 'yolov8n.pt'
batch = 64 

model = YOLO(weight)
data_training =  "/kaggle/working/crosswalk_dataset.yaml"
project = proyect_results_path


In [ ]:
print(type(model))

In [ ]:
%pip install -U ipywidgets

In [ ]:
 model.train(data=data_training,
             epochs=epochs,
             imgsz=512,
             batch=batch,
             cache=True,
             patience=20,
             project=project,
             device=[0, 1],
             # Esto especifica el uso de las GPUs 0 y 1
            )

#Crear el YAML para ver los resultados del test

In [ ]:
yaml_content = """
path: /kaggle/working/dataset

train: /kaggle/working/
val: test/images


nc: 1

names:
     - crosswalk
"""

# Guardar el archivo
with open('/kaggle/working/crosswalk_test.yaml', 'w') as file:
    file.write(yaml_content)

print("Archivo .yaml creado con éxito.")

In [ ]:
from ultralytics import YOLO

weight = '/kaggle/working/training_results/train3/weights/best.pt'
model = YOLO(weight)


In [ ]:
data_test = "/kaggle/working/crosswalk_test.yaml"
# Validar el modelo y especificar el directorio de salida
val_project = '/kaggle/working/validation_results'
val_name = 'crosswalk_val'
metrics = model.val(data=data_test, project=val_project, name=val_name)


In [ ]:
import shutil
from pathlib import Path
import os

# Ruta donde YOLO guarda los resultados por defecto
default_val_path = Path('runs/val')

# Nuevo directorio donde quieres mover los resultados
new_val_path = Path('/kaggle/working/validation_results')

# Verificar si el directorio de resultados por defecto existe
if default_val_path.exists() and default_val_path.is_dir():
    # Crear el nuevo directorio si no existe
    new_val_path.mkdir(parents=True, exist_ok=True)
    
    # Mover los resultados de validación al nuevo directorio
    shutil.move(str(default_val_path), str(new_val_path))
    print(f"Resultados de la validación movidos a {new_val_path}")
else:
    print(f"El directorio {default_val_path} no existe. Verifica si la validación se ejecutó correctamente.")

In [ ]:
import os
import zipfile

def create_zip_from_folders(folders, output_zip_path):
    """
    Crea un archivo .zip de las carpetas especificadas.

    Args:
        folders (list): Lista de rutas de carpetas a incluir en el .zip.
        output_zip_path (str): Ruta del archivo .zip de salida.
    """
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for folder in folders:
            for root, dirs, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, os.path.join(folder, '..'))
                    zipf.write(file_path, arcname)

# Ejemplo de uso:
folders_to_zip = [
    '/kaggle/working/training_results',
    '/kaggle/working/validation_results',
    '/kaggle/working/runs',
    '/kaggle/working/yolov8n.pt'
    
]
output_zip_path = '/kaggle/working/crosswalk_results.zip'

create_zip_from_folders(folders_to_zip, output_zip_path)

print(f"Archivo .zip creado en {output_zip_path}")
